In [1]:
import numpy as np
import pandas as pd
import os

# create X_train, y_train, X_val, y_val, X_test, y_test randomly without anything

X_train = np.random.rand(100, 10)
# Y are binary
y_train = np.random.randint(2, size=100)
X_val = np.random.rand(30, 10)
y_val = np.random.randint(2, size=30)
X_test = np.random.rand(30, 10)
y_test = np.random.randint(2, size=30)



In [2]:
# Keras tuner
import tensorflow as tf
from tensorflow import keras

In [3]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [4]:
import keras_tuner as kt

In [23]:
import kerastuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

input_shape = X_train.shape[1]  # Number of features from your training data

# Define a model-building function for KerasTuner
def build_model(hp):
    model = Sequential()
    # First Dense layer
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=256, step=96),
                    activation='relu', input_shape=(input_shape,)))  # input shape inferred
    # Dropout layer after the first Dense layer
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.25)))
    
    # Second Dense layer
    model.add(Dense(units=hp.Int('units2', min_value=32, max_value=256, step=96),
                    activation='relu'))  
    # Dropout layer after the second Dense layer
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.25)))

    model.add(Dense(10, activation='softmax'))  # Example output layer

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the tuner by passing the build_model function and specifying the objective
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',  # Assuming you have a validation set to evaluate on
    max_epochs=10,  # Maximum number of epochs to train one model. Adjust as necessary.
    factor=3,  # Reduction factor for the number of epochs and number of models for each bracket.
    #directory='my_dir',
    #project_name='hyperband_intro'
)

# Display search space summary
tuner.search_space_summary()

# Execute the search
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

# Get the best hyperparameters and rebuild the model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"{best_hps}")
model = build_model(best_hps)

# You might want to train and evaluate here or just directly get the summary
model.summary()


Trial 28 Complete [00h 00m 01s]
val_accuracy: 0.44999998807907104

Best val_accuracy So Far: 0.699999988079071
Total elapsed time: 00h 00m 23s
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                352       
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 128)               4224      
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 5,866
Trainable params: 5,866

In [25]:
# Show parameters of model
print(model.get_config())

# add early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=3)

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[early_stopping_monitor])

# Evaluate the model
model.evaluate(X_test, y_test)

print(model.get_config())

{'name': 'sequential_1', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 10), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'dense_3_input'}}, {'class_name': 'Dense', 'config': {'name': 'dense_3', 'trainable': True, 'batch_input_shape': (None, 10), 'dtype': 'float32', 'units': 32, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}, {'class_name': 'Dropout', 'config': {'name': 'dropout_2', 'trainable': True, 'dtype': 'float32', 'rate': 0.5, 'noise_shape': None, 'seed': None}}, {'class_name': 'Dense', 'config': {'name': 'dense_4', 'trainable': True, 'dtype': 'float32', 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'conf

In [20]:


# print all
print(best_model.summary())
print(best_model.optimizer)
print(best_model.loss)
# learning rate
print(best_model.optimizer.get_config()['learning_rate'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 19,210
Trainable params: 19,210
Non-trainable params: 0
_________________________________________________________________
None
sparse_categorical_crossentropy
0.001


In [21]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 96, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.25, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 96, 'sampling': 'linear'}
dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.25, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}
